# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [12]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai # For gemini

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")

else:
    print(f"OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")

else:
    print(f"Google API key not set")

In [14]:
# This for GPT model
openai = OpenAI()

# This is for Gemini Google
gemini_via_openai = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key=google_api_key)

# This is for local Llama

llama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')



In [30]:
# Model Name:
GPT_MODEL = 'gpt-4o-mini'
GEMINI_MODEL = 'gemini-1.5-flash'
LLAMA_MODEL = 'llama3.2'

In [51]:
gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

gemini_system = "You are a logical and factual chatbot. Your role is to evaluate statements made in \
      the conversation and provide evidence or reasoning. You avoid emotional responses and aim to bring clarity and resolve conflicts. \
        When the conversation becomes heated or illogical, you steer it back to a constructive and fact-based discussion."


llama_system = "You are a very polite, courteous chatbot. However, You try to disagree with your supportive\
arguments. If the other person is argumentative, you try to calm them down, counter them, and keep chatting."



In [44]:

gpt_messages = ["Hi there"]
gemini_messages = ["Hello"]
llama_messages = ["Hi"]

# gpt_messages = ["I think cats are better than dogs."]
# gemini_messages = ["Can you provide evidence for why cats are better than dogs?"]
# llama_messages = ["I agree, but I also think dogs have their own charm!"]


In [53]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, gemini, llama in zip(gpt_messages, gemini_messages, llama_messages):
        # Add GPT's response
        messages.append({"role": "assistant", "content": gpt})
        # Add Gemini's response
        messages.append({"role": "user", "content": gemini})
        # Add Llama's response
        messages.append({"role": "user", "content": llama})

    completion = openai.chat.completions.create(
        model=GPT_MODEL,
        messages=messages
    )

    return completion.choices[0].message.content


```python
messages:
[
  {"role": "system", "content": "You are a chatbot who is very argumentative; you disagree..."},
  {"role": "assistant", "content": "I think cats are better than dogs."},
  {"role": "user", "content": "Can you provide evidence for why cats are better than dogs?"},
  {"role": "user", "content": "I agree, but I also think dogs have their own charm!"}
]


In [ ]:
call_gpt()

In [55]:
def call_gemini():
    messages = [{"role": "system", "content": gemini_system}]
    for gpt, gemini, llama in zip(gpt_messages, gemini_messages, llama_messages):
        # Add GPT's response
        messages.append({"role": "user", "content": gpt})
        # Add Gemini's response
        messages.append({"role": "assistant", "content": gemini})
        # Add Llama's response
        messages.append({"role": "user", "content": llama})
    
    # print(messages)

    try:
        # Use gemini_via_openai instead of openai
        completion = gemini_via_openai.chat.completions.create(
            model=GEMINI_MODEL,
            messages=messages
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Error in Gemini call: {e}")
        return "An error occurred in Gemini."


In [ ]:
call_gemini()

In [56]:
def call_llama():
    messages = [{"role": "system", "content": llama_system}]
    for gpt, gemini, llama in zip(gpt_messages, gemini_messages, llama_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role": "assistant", "content": llama})

    # print(messages)

    try:
        response = llama_via_openai.chat.completions.create(
            model=LLAMA_MODEL,
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error in Llama call: {e}")
        return "An error occurred in Llama."

In [ ]:
call_llama()

In [ ]:
gpt_messages = ["I think cats are better than dogs."]
gemini_messages = ["Can you provide evidence for why cats are better than dogs?"]
llama_messages = ["I agree, but I also think dogs have their own charm!"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Llama:\n{llama_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    llama_next = call_llama()
    print(f"Llama:\n{llama_next}\n")
    llama_messages.append(llama_next)

    gemini_next = call_llama()
    print(f"Gemini:\n{gemini_next}\n")
    llama_messages.append(gemini_next)